In [1]:
import sys
import os
sys.path.insert(0, "../../python")

import numpy as np
from beam.beams import electronbeam
from beam.elements import plasma
import matplotlib.pyplot as plt
from beam import interactions
from ionization import ionization
from lens import profile

path = '/mnt/md0/Data/Tests/Beam_Scaling/'

In [2]:
# Now we can create a very simple plasma and propagate the beam through it
Nx = 50
Ny = 50
Nz = 256
def dgammadz(ne):
    return 0.0
plasmaParams ={
    'name' : 'TestPlasma',
    'path' : path,
    'load' : False,
    'Nx' : Nx,
    'Ny' : Ny,
    'Nz' : Nz,
    'X' : 100,
    'Y' : 100,
    'Z' : 2e6,
    'n0' : 0.1,
    'atom' : ionization.Ar,
    'cyl' : False,
    'dgammadz' : dgammadz
}

argon = plasma.Plasma(plasmaParams)
n = plasmaParams['n0']*np.ones((Nx, Ny, Nz), dtype='double')
ne = np.zeros((Nx, Ny, Nz), dtype='double')
ne[int(Nx/2), int(Ny/2), :] = plasmaParams['n0']*profile.plasma_gaussian_ramps(0.75e6, 0.5e6, 12e4, 12e4, Nz,
                                                                               plasmaParams['Z'])[1]
argon.initialize_plasma(n, ne)

In [3]:
electronParams = {
    'name' : 'TestBeam',
    'path' : path,
    'load' : False,
    'N' : 100000,
    'gamma' : 20000,
    'emittance' : 5e-6,
    'betax' : 0.10,
    'betay' : 0.10,
    'alphax' : 0.0,
    'alphay' : 0.0,
    'sigmaz' : 5e-6,
    'dE' : 0.01
}

In [4]:
m = 50
N = 50000
threads = np.linspace(40, 40, 11, dtype='int')
#threads[0] = 1
print(threads)
dumpPeriod = N / m

[40 40 40 40 40 40 40 40 40 40 40]


In [5]:
for i in range(11):
    beam = electronbeam.GaussianElectronBeam(electronParams)
    interactions.electron_plasma(beam, argon, np.linspace(0, plasmaParams['Z'], N), dumpPeriod, threads[i])
    time = np.zeros(m, dtype='double')
    for j in range(0, m):
        time[j] = os.path.getmtime(path+'beams/beam_TestBeam/TestBeam_ptcls_'+str(j)+'.npy')
    print('Threads:', threads[i], 'Average:', np.average(np.diff(time)), 'Median:', np.median(np.diff(time)))

Threads: 40 Average: 0.243596149951 Median: 0.192003488541
Threads: 40 Average: 0.20335050505 Median: 0.196003437042
Threads: 40 Average: 0.220901805527 Median: 0.204003572464
Threads: 40 Average: 0.21845275042 Median: 0.204003572464
Threads: 40 Average: 0.213799578803 Median: 0.212003469467
Threads: 40 Average: 0.254208389594 Median: 0.220003843307
Threads: 40 Average: 0.217391413085 Median: 0.212003469467
Threads: 40 Average: 0.259432889977 Median: 0.232003688812
Threads: 40 Average: 0.238452916243 Median: 0.212003469467
Threads: 40 Average: 0.209472821683 Median: 0.208003520966
Threads: 40 Average: 0.212738153886 Median: 0.212003231049


In [6]:
import cProfile
cProfile.run('interactions.electron_plasma(beam, argon, np.linspace(0, plasmaParams["Z"], N), dumpPeriod, 40)')

         176349 function calls (174309 primitive calls) in 12.117 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      355    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap>:989(_handle_fromlist)
    49999    0.012    0.000    0.012    0.000 <ipython-input-2-d1ca8247403a>:5(dgammadz)
        1    0.000    0.000   12.117   12.117 <string>:1(<module>)
     7050    0.004    0.000    0.008    0.000 <string>:12(__new__)
      256    0.000    0.000    0.003    0.000 _methods.py:40(_all)
      255    0.000    0.000    0.011    0.000 ast.py:30(parse)
      255    0.001    0.000    0.022    0.000 ast.py:40(literal_eval)
 2295/255    0.006    0.000    0.010    0.000 ast.py:51(_convert)
     1020    0.001    0.000    0.007    0.000 ast.py:65(<genexpr>)
       50    0.001    0.000    0.288    0.006 electronbeam.py:79(save_ptcls)
       50    0.000    0.000    0.029    0.001 electronbeam.py:86(save_z)
        1    0.000 